In [13]:
#IMPORTING SOME LIBRARY

import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
from collections import defaultdict
import warnings
import time
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import json
import requests

from datetime import datetime, date, timedelta

In [14]:
#Config chart

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'

In [15]:
url = "https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1m"
r = requests.get(url)
klines = np.array(r.json())
klines

array([['1619400480000', '2416.18000000', '2421.66000000', ...,
        '353.99025000', '856026.00319940', '0'],
       ['1619400540000', '2416.81000000', '2418.98000000', ...,
        '257.18111000', '621705.92008800', '0'],
       ['1619400600000', '2416.96000000', '2423.64000000', ...,
        '373.79736000', '905107.85298880', '0'],
       ...,
       ['1619430300000', '2448.18000000', '2454.69000000', ...,
        '1600.69923000', '3922122.92880010', '0'],
       ['1619430360000', '2453.53000000', '2464.00000000', ...,
        '1061.88525000', '2610463.49018400', '0'],
       ['1619430420000', '2463.58000000', '2464.75000000', ...,
        '413.50639000', '1018496.07843430', '0']], dtype='<U21')

In [17]:
def binance_to_df(klines):
    df = pd.DataFrame(klines.reshape(-1,12),dtype=float, columns = ('t',
                                                                    'o',
                                                                    'h',
                                                                    'l',
                                                                    'c',
                                                                    'v',
                                                                    'Close time',
                                                                    'Quote asset volume',
                                                                    'Number of trades',
                                                                    'Taker buy base asset volume',
                                                                    'Taker buy quote asset volume',
                                                                    'Ignore'))

    df['t'] = pd.to_datetime(df['t'], unit='ms')


    return df

prices = binance_to_df(klines)
prices.head()

,t,o,h,l,c,v,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2021-04-26 01:28:00,2416.18,2421.66,2415.39,2416.81,685.21020,1.619401e+12,1.657184e+06,840.0,353.99025,8.560260e+05,0.0
1,2021-04-26 01:29:00,2416.81,2418.98,2415.99,2416.86,509.36885,1.619401e+12,1.231319e+06,902.0,257.18111,6.217059e+05,0.0
2,2021-04-26 01:30:00,2416.96,2423.64,2416.96,2422.48,792.93056,1.619401e+12,1.919950e+06,960.0,373.79736,9.051079e+05,0.0
3,2021-04-26 01:31:00,2422.74,2432.66,2421.46,2431.94,2338.77610,1.619401e+12,5.681654e+06,2510.0,1345.47740,3.267915e+06,0.0
4,2021-04-26 01:32:00,2431.94,2439.00,2431.50,2432.29,3450.42473,1.619401e+12,8.404927e+06,4068.0,1567.29247,3.817936e+06,0.0


In [ ]:
def resample(stock_data, rate):
    stock_data = stock_data.resample(rate, on='t',closed='right', label='right').agg({'o': 'first',
                                                                                                 'h': 'max',
                                                                                                 'l': 'min',
                                                                                                 'c': 'last'}).dropna()
    return stock_data
